# Описание
Представим, что нам необходимо провести анализ рынка найма ..........

## Задачи
1. Подсчитать среднюю заработную плату "на руки" для каждого региона/города/страны, представленных в последних 2000 вакансий.
2. Составить ТОП-10 самых востребованных профессий в городах миллионниках.
3. Определить долю вакансий с отсутствующими данными о заработной плате.
4. Определить доля вакансий с указанным полным рабочим днем.
5. Выделить ТОП-10 ключевых навыков для менеджерских профессий.

После выполенения вышеперечисленных пунктов, необходимо сделат выводы и попробовать обосновать полученный результат.

## Решение

### Импортирование модулей и подготовка функций
На данном этапе нам необходимо имортировать 3 модуля:
* request - для осуществления доступа к данным по API;
* json - для скачивания данных в .json формате;
* collections - для удобного хранения и подсчета некоторых объектов;
* typing - для аннотаций, создаваемых функций.

In [1]:
import json
import requests
from collections import Counter
from typing import Iterable

Далее мы создаем функцию, для корректного отображения результатов.

In [2]:
def choose_plural(amount: int, declensions: Iterable[str]) -> str:
    """Функция, которая принимает количество и все доступные склонения слова,
    а возвращает правильный вариант склонения слова
    """
    mnemo = {'1': 0, '2': 1, '3': 1, '4': 1, '5': 2,
             '6': 2, '7': 2, '8': 2, '9': 2, '0': 2,
             '11': 2, '12': 2, '13': 2, '14': 2
            }
    a = str(amount)
    return declensions[mnemo[a[-2:] if 10 < int(a[-2:]) < 15 else str(amount)[-1]]]

Также создадим рекурсивную функцию, которая поможет нам достать из json-файла с регионами все города и области в одномерный список.

In [3]:
def get_countries_with_cities(data: list[dict]) -> dict[str: set]:
    def inner_rec(current: Iterable, country: str) -> None:        
        for area in current:
            _countries[country].add(area['name'])

            if area['areas'] != []:
                inner_rec(area['areas'], country)


    _countries = {}
    for country in data:
        _countries[country['name']] = {country['name']}
        if country['areas'] != []:
            inner_rec(country['areas'], country['name'])
            
    _countries |= {
        country: {country}
        for country in _countries.pop('Другие регионы') - {'Другие регионы'}
        }
    
    return _countries

### Загрузка данных

* Прежде всего, нам необходимо загрузить 20 страниц (максимально доступное количество), котрые содержат краткое описание каждой вакансии. Для этого удобно использовать лямбда-функцию, которая будет принимать порядковй номер страницы (индексация начинается с нуля) и возвращать URL необходимой страницы.
* Далее нам необходимо скачать и получить доступ к подробной информации о каждой вакансии, для этого так же удобно использовать лямбда-функцию, которая будет принимать содержимое поля 'id' конкретной вакансии и возвращать URL страницы с подробной информацией о данной вакансии.
* Следующим шагом мы должны создать словарь с текущим курсом валют, которые доступны для указания в вакансии. Здесь я прибегнул к выбору API, предоставляемого сервисом [fixer.io](https://fixer.io/), так как сторонние модули (forex-python и currency_converter) либо несовместимы с версией Python 3.11.4, либо не предоставляют возможность конвертации специфических валют, либо на момент написание проекта имеют неразрешенные проблемы внутри модуля.
* Также, мы должны получить информацию о регионах, странах и городах, чтобы в дальнейшем сегментировать вакансии в соответсвии с 1-м и 2-м пунктами.

In [4]:
page_api = lambda i: f'https://api.hh.ru/vacancies?per_page=100&page={i}&pages=20&locale=RU'
vacancy_api = lambda id: f'https://api.hh.ru/vacancies/{id}'
currency_api = 'http://data.fixer.io/api/latest?access_key=8ec671c059abd588359328e0199e0a63&symbols=BYN,RUB,KZT,UZS,USD,KGS,GEL,AZN,UAH'
areas_api = 'https://api.hh.ru/areas?locale=RU'

Создаем итератор, загружающий страницы с кратким описанием каждой вакансии

In [5]:
data = (
    json.loads(requests.get(page_api(i)).text)['items']
    for i in range(20)
    )

Создаем итератор, загружающий с каждой страницы подробное описание каждой вакансии

In [6]:
vacancies = (
    json.loads(requests.get(vacancy_api(vacancy['id'])).text)
    for page in data
    for vacancy in page
    )

Получаем курсы необходимых валют относительно евро (валюта/евро). В качестве валют на HH.ru можно указать: RUR (российский рубль), BYR (белорусский рубль), KZT (казахстанский тенге), UZS (узбекский сум), USD (американский доллар), EUR (евро), KGS (киргизский сом), GEL (грузинский лари), AZN (азербайджанский манат) и UAH (украинская гривна).

In [7]:
currencies = json.loads(requests.get(currency_api).text)['rates']

Скачиваем данные о странах, городах и регионах, которые хранятся в json-файле в виде **списка словарей, котрые в свою очередь могут иметь списки словарей и так далее**. Таким образом, нам понадобится использовать рекурсивную функцию, чтобы преобразовать данные к линейному словарю множеств, то есть привести их к виду:
```json
{
    'страна1': {'страна1', 'область1', 'город1', ...},
    'страна2': {'страна2', 'область2', 'город2', ...},
    ...
}
```
Почему в значения мы помещаем области и страны? Потому, что HH.ru допускает указание в вакансии области или страны, без уточнения города, в таком случае **HH.ru устанавливает для вакансии общий код 113**, что и является Россией в целом. Также, для некоторых стран отсутствует возможность указать конкретный город, как например для ОАЭ или США, и поэтому **HH.ru относит их к коду 1001**, который называется 'Другие регионы'.

Таким образом, для корректного вычисления средней з/п по странам, а также корректного вывода средней з/п по городам России, нам необходимо прибегнуть к данному решению.

In [8]:
countries_and_cities = get_countries_with_cities(json.loads(requests.get(areas_api).text))

### Подготовка переменных
* **salary_by_area** - это словарь для хранения названий **городов/стран/регионов** и списков зарплат из вакансий;
* **grouped_avg_salaries** - список групп, где 1-й словарь - все города России, 2-й - города миллионники, а 3-й - страны;
* **millionaire_cities** - множество всех городов-миллионников на момент прведения анализа (октябрь 2023 года);
* **num_of_vacs** - общее кол-во вакансий;
* **ommited_salary_vacs** - счетчик вакансий с неуказанной з/п;
* **full_time_vacs** - счетчики вакансий с полным рабочим днем;
* **managemental_vacs** - счетчик менеджерских вакансий;
* **popular_professions** - множество, содержащее профессии, на которые ищут людей из ТОП-10 городов;
* **key_skills** - множество, содержащее ключевые навыки, требуемые для менеджеров.

In [9]:
salary_by_area = {}
grouped_avg_salaries = [{}, {}, {}]
millionaire_cities = {
    'Москва', 'Санкт-Петербург', 'Екатеринбург', 'Новосибирск',
    'Казань', 'Нижний Новгород', 'Челябинск', 'Красноярск',
    'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск',
    'Краснодар', 'Воронеж', 'Волгоград', 'Пермь'
}
num_of_vacs = 2000
ommited_salary_vacs = full_time_vacs = managmental_vacs = 0
popular_professions = Counter()
key_skills = Counter()

Для того, чтобы подготовить наш словарь для конвертации валют в рубль, нам необходимо выполнить следующие действия:
1. **Добавить евро**, так как данная валюта на данный момент отсутствует;
2. Привести словарь к виду **{тикер валюты: отношение валюты к рублю}**;
3. **Переименовать тикеры российского и белорусского рублей**, так как на HH.ru используются иные тикеры, а исходные тикеры удаляем

In [10]:
currencies['EUR'] = 1

currencies = {k: currencies.copy()['RUB'] / v for k, v in currencies.copy().items()}

currencies['RUR'], currencies['BYR'] = currencies.pop('RUB'), currencies.pop('BYN')

### Обработка данных
Следующим этапом приступаем к обработке данных. На данном этапе мы должны:
1. Сопоставить каждому региону среднюю зарплату по вакансиям из последних 2000;
2. Составить ранжированный список из ТОП-10 требуемых профессий в городах-миллионниках;
3. Вычислить количество вакансий с неуказанной з/п;
4. Вычислить количество вакансий с полнодневной рабочей занятостью;
5. Составить ранжированный список из ТОП-10 требуемых ключевых навыков для менеджерских профессий.

1. Для того, чтобы осуществить пункт №1, нам необходимо разбить задачу на 2 шага, то есть на 2 цикла, первый из которых будет добавлять все заработные платы для каждого города или страны в общий словарь, а второй - вычислять среднюю заработную плату и распределять регионы по соответствующим словарям в списке 'grouped_avg_salaries'. Разберем 1-й цикл:
* В первом цикле мы должны наполнить словарь 'salary_by_area' таким образом, чтобы он принял вид:
```json
{
    'ГОРОД_1': ['ЗП_1', 'ЗП_2', ...],
    'ГОРОД_2': ['ЗП_1', 'ЗП_2', ...],
    'СТРАНА_1': ['ЗП_1', 'ЗП_2', ...]
}
```
* Стоит учесть, что поле 'salary' может принимать значение None, и это должно проверяться внутри цикла. Таким образом, если поле 'salary' соответствует значению None, мы можем увеличить счетчик вакансий с неуказанной зарплатой на единицу, то есть заодно мы в этом цикле решаем пункт №3. Если з/п указана, мы можем перейти к следующему шагу.
* Если зарплата указана, она может быть представлена как в виде диапазона, так и в виде одной из границ, поэтому нам необходимо проверить это. Если з/п представленна диапазоном, мы ищем среднее между границами, в противном случае берем в качестве средней з/п одну из границ.
* Обратим внимание на то, что з/п может быть указана в разных валютах, поэтому мы должны учесть это и привести ее в рублевый эквивалент (они находятся в словаре 'currencies').
* Далее мы должны проверить включен ли в з/п НДФЛ и если нет, то умножить з/п на 1 - 0,13.

2. Вместе с тем, в текущем цикле можно выполнить 2-й пункт:
* Мы можем проверить является ли город указанный в вакансии миллионником и если да, то добавляем все профессиональные роли из указанной вакансии в соответствующее множество.

3. По совместительству в данном цикле будет удобно выполнить 4-й пункт:
* Для выполнения 4-го пункта мы можем проверить указано ли поле 'schedule' в вакансии и если так, то прибавить к счетчику единицу, если указаный формат соответствует полному дню.

4. Также в данном цикле мы можем сделать и пункт №5:
* Создадим перменную 'roles' и положим в нее строку, состоящую из всех профессиональных ролей (для 1 вакансии их может быть несколько), переведенных в нижний регистр.
* Далее мы проверим данную строку на вхождение в нее корней 'менедж' и 'manage' и если один из них присутствует, тогда мы добавляем в словарь-каунтер все ключевые навыки, необходимые для данной менеджерской вакансии.

In [11]:
for vacancy in vacancies:
    area = vacancy['area']['name'] # Сохраняем для удобства

    # Проверяем, указана ли з/п, если нет, то увеличиваем счетчик, а если да, то прикрепляем ее к региону
    if vacancy['salary'] is not None:
        salary_data = vacancy['salary'] # Сохраняем для удобства
        ratio = currencies[salary_data['currency']] # Множитель для перевода валюты в рубли
        payroll = (salary_data['from'], salary_data['to']) # Зарплатная вилка
        tax = (0, 0.13)[salary_data['gross']] # Налог, если оне не включет в указанную з/п

        if all(payroll):
            avg_salary = sum(payroll) / 2
        else:
            avg_salary = payroll[0] or payroll[1]

        real_avg_salary = (1 - tax) * avg_salary * ratio
        salary_by_area.setdefault(area, []).append(real_avg_salary)
    else:
        ommited_salary_vacs += 1

    # Проверяем является ли город миллионником и если да, то добавляем профессиональные роли из этой вакансии
    if area in millionaire_cities:
        popular_professions += Counter(role['name'] for role in vacancy['professional_roles'])

    # Проверяем указал ли работодатель расписание работы и если так, то увеличиваем счетчик если формат полнодневный
    if vacancy['schedule'] is not None:
        full_time_vacs += (vacancy['schedule']['name'] == 'Полный день')

    # Проверяем является ли вакансия менеджерской и если так, то добавляем в словарь навыки
    roles = ' '.join(role['name'].lower() for role in vacancy['professional_roles'])
    if 'менедж' in roles or 'управл' in roles or 'manag' in roles:
        managmental_vacs += 1
        skills = vacancy['key_skills']
        key_skills += Counter(skill['name'] for skill in skills)

Удаляем из множества профессий 'Другое'. Данное название ассоциировано с профессиями, которые **не попали под классификацию HH.ru**

In [12]:
del popular_professions['Другое']

Теперь, когда все вакансии обработаны, мы "схлопываем" списки в **среднюю з/п по соответствующему региону** в каждом из словарей. А также распределяем получившиеся значения по соответствующим словарям.

Важно отметить, что если регион не принадлежит России, то необходимо добавить среднюю зарплату по данному городу в список по ключу страны, к которой пренадлежит этот город.

In [13]:
for area, salaries in salary_by_area.items():
    avg_salary = round(sum(salaries) / len(salaries), 2)

    if area in millionaire_cities:
        grouped_avg_salaries[1][area] = avg_salary
    elif area in countries_and_cities['Россия']:
        grouped_avg_salaries[0][area] = avg_salary
    else:
        for country, cities in countries_and_cities.items():
            if area in cities:
                grouped_avg_salaries[2].setdefault(country, []).append(avg_salary)
                break

Добавляем в словарь стран среднюю зарплату по России. А также окончательно рассчитываем среднюю зарплату по каждой стране

In [14]:
grouped_avg_salaries[2]['Россия'] = grouped_avg_salaries[0].values()

for area, salaries in grouped_avg_salaries[2].items():
    avg_salary = round(sum(salaries) / len(salaries), 2)
    grouped_avg_salaries[2][area] = avg_salary

Как было сказано ранее, в некоторых вакансиях указана Россия в целом, а не конкретный город, поэтому для корректности удалим соответствующую пару из словаря городов России.

In [15]:
del grouped_avg_salaries[0]['Россия']

## Результаты

### Пункт №1

Ниже мы сортируем словарь **с городами России** по убыванию средней заработной платы и выводим получившиеся результаты.

In [16]:
for area, avg_salary in sorted(grouped_avg_salaries[0].items(), key=lambda x: x[1], reverse=True):
    print(f'Средняя зарплата в городе {area} составляет {avg_salary} руб.')

Средняя зарплата в городе Улан-Удэ составляет 565500.0 руб.
Средняя зарплата в городе Шира составляет 165000.0 руб.
Средняя зарплата в городе Нижний Тагил составляет 160080.0 руб.
Средняя зарплата в городе Зеленоград составляет 160000.0 руб.
Средняя зарплата в городе Щербинка составляет 135000.0 руб.
Средняя зарплата в городе Мытищи составляет 134850.0 руб.
Средняя зарплата в городе Алексин составляет 130500.0 руб.
Средняя зарплата в городе Мурманск составляет 127666.67 руб.
Средняя зарплата в городе Альметьевск составляет 125000.0 руб.
Средняя зарплата в городе Владивосток составляет 118400.0 руб.
Средняя зарплата в городе Южно-Сахалинск составляет 110925.0 руб.
Средняя зарплата в городе Сочи составляет 107500.0 руб.
Средняя зарплата в городе Химки составляет 98750.0 руб.
Средняя зарплата в городе Симферополь составляет 98000.0 руб.
Средняя зарплата в городе Новокузнецк составляет 96845.56 руб.
Средняя зарплата в городе Калининград составляет 95500.0 руб.
Средняя зарплата в городе Тул

Ниже мы сортируем словарь **с городами-миллионниками** по убыванию средней заработной платы и выводим получившиеся результаты.

In [17]:
for area, avg_salary in sorted(grouped_avg_salaries[1].items(), key=lambda x: x[1], reverse=True):
    print(f'Средняя зарплата в городе {area} составляет {avg_salary} руб.')

Средняя зарплата в городе Москва составляет 118805.62 руб.
Средняя зарплата в городе Санкт-Петербург составляет 106874.63 руб.
Средняя зарплата в городе Волгоград составляет 101914.03 руб.
Средняя зарплата в городе Екатеринбург составляет 94012.95 руб.
Средняя зарплата в городе Новосибирск составляет 80194.39 руб.
Средняя зарплата в городе Красноярск составляет 79821.67 руб.
Средняя зарплата в городе Ростов-на-Дону составляет 78720.77 руб.
Средняя зарплата в городе Краснодар составляет 77353.23 руб.
Средняя зарплата в городе Казань составляет 73431.18 руб.
Средняя зарплата в городе Нижний Новгород составляет 71205.38 руб.
Средняя зарплата в городе Самара составляет 64573.44 руб.
Средняя зарплата в городе Уфа составляет 63453.93 руб.
Средняя зарплата в городе Челябинск составляет 60963.33 руб.
Средняя зарплата в городе Пермь составляет 56972.5 руб.
Средняя зарплата в городе Воронеж составляет 50970.17 руб.
Средняя зарплата в городе Омск составляет 48407.1 руб.


Ниже мы сортируем словарь **со странами** по убыванию средней заработной платы и выводим получившиеся результаты.

In [18]:
for area, avg_salary in sorted(grouped_avg_salaries[2].items(), key=lambda x: x[1], reverse=True):
    print(f'Средняя зарплата в стране {area} составляет {avg_salary} руб.')

Средняя зарплата в стране США составляет 560849.82 руб.
Средняя зарплата в стране Испания составляет 273559.48 руб.
Средняя зарплата в стране ОАЭ составляет 250000.0 руб.
Средняя зарплата в стране Южная Корея составляет 200000.0 руб.
Средняя зарплата в стране Таиланд составляет 150000.0 руб.
Средняя зарплата в стране Китай составляет 108750.0 руб.
Средняя зарплата в стране Грузия составляет 98822.03 руб.
Средняя зарплата в стране Казахстан составляет 72331.77 руб.
Средняя зарплата в стране Кыргызстан составляет 71948.37 руб.
Средняя зарплата в стране Россия составляет 67017.65 руб.
Средняя зарплата в стране Узбекистан составляет 60708.25 руб.
Средняя зарплата в стране Украина составляет 39726.86 руб.
Средняя зарплата в стране Беларусь составляет 32389.33 руб.


### Пункт №2

Далее сортируем ТОП-10 самых востребованных профессий в городах миллионниках и выводим их в порядке убывания спроса.

In [19]:
for prof, amount in sorted(popular_professions.items(), key=lambda x: x[::-1], reverse=True)[:10]:
    decl = choose_plural(amount, ('вакансия', 'вакансии', 'вакансий'))
    print(f'{prof}: {amount} {decl}')

Водитель: 165 вакансий
Бухгалтер: 85 вакансий
Менеджер по продажам, менеджер по работе с клиентами: 67 вакансий
Программист, разработчик: 48 вакансий
Администратор: 48 вакансий
Продавец-консультант, продавец-кассир: 42 вакансии
Секретарь, помощник руководителя, ассистент: 37 вакансий
Упаковщик, комплектовщик: 29 вакансий
Делопроизводитель, архивариус: 27 вакансий
Дизайнер, художник: 26 вакансий


### Пункт №3

In [20]:
print(f'Из воследних 2000 вакансий доля тех, где не указана зарплата: {100 * ommited_salary_vacs / num_of_vacs}%')

Из воследних 2000 вакансий доля тех, где не указана зарплата: 16.05%


### Пункт №4

In [21]:
print(f'Из воследних 2000 вакансий доля тех, где указан полный рабочий день: {100 * full_time_vacs / num_of_vacs}%')

Из воследних 2000 вакансий доля тех, где указан полный рабочий день: 77.6%


### Пункт №5

Ниже представлены ТОП-10 ключевых навыков, требуемых для менеджерских профессий.

In [22]:
for skill, amount in sorted(key_skills.items(), key=lambda x: x[1], reverse=True)[:10]:
    decl = choose_plural(amount, ('вакансии', 'вакансиях', 'вакансиях'))
    print(f'Навык "{skill}" указан в {amount} {decl} ({round(100 * amount / managmental_vacs, 2)}%)')

Навык "Активные продажи" указан в 44 вакансиях (19.21%)
Навык "Грамотная речь" указан в 40 вакансиях (17.47%)
Навык "Телефонные переговоры" указан в 40 вакансиях (17.47%)
Навык "Деловое общение" указан в 34 вакансиях (14.85%)
Навык "Деловая переписка" указан в 31 вакансии (13.54%)
Навык "Навыки продаж" указан в 30 вакансиях (13.1%)
Навык "Работа в команде" указан в 26 вакансиях (11.35%)
Навык "Поиск и привлечение клиентов" указан в 26 вакансиях (11.35%)
Навык "Заключение договоров" указан в 26 вакансиях (11.35%)
Навык "Ведение переговоров" указан в 22 вакансиях (9.61%)


## Выводы